# Merge Nodes

One issue with SemmedDB (or the UMLS Metathesaurus in general) is that the CUIs are too granular in detail.  

Take for example Imatinib Mesylate.  The following concepts are all found within SemmedDB:

| UMLS CUI | Concept Name      |
|----------|-------------------|
| C0939537 | Imatinib mesylate |
| C0385728 | CGP 57148         |
| C1097576 | ST 1571           |
| C0935987 | Gleevec           |
| C0906802 | STI571            |
| C0935989 | imatinib          |

However, all of these concepts describe the same chemical structure.  Luckily, all of these UMLS CUIs can be cross-referenced to just 1 MeSH Descriptor ID: `D000068877`.  This will allow us to merge these concept within the network.

Diseases have similar issues, however, they are a little less straightforward.  A similar, yet more complex approach will be used for thier combination.

In [1]:
from tqdm import tqdm
from collections import defaultdict
from collections import Counter
from queue import Queue
from itertools import chain
import pandas as pd
import pickle

import sys
sys.path.append('../../hetnet-ml/src')
import graph_tools as gt

sys.path.append('../tools/')
import load_umls

## 1. Import the DrugCentral info for Gold Standard and Add Compound Names

In [2]:
rels = pd.read_csv('../data/drugcentral_rel_06212018.csv')
rels.head(2)

,id,struct_id,concept_id,relationship_name,concept_name,umls_cui,snomed_full_name,cui_semantic_type,snomed_conceptid
0,173432,965,40249340,indication,Malignant tumor of breast,C0006142,Malignant tumor of breast,T191,254837009.0
1,173433,318,21000716,indication,Gout,C0018099,Gout,T047,90560007.0


In [3]:
dc_ids = pd.read_csv('../data/drugcentral_ids_06212018.csv')
dc_ids.head(2)

,id,identifier,id_type,struct_id,parent_match
0,1214712,D11040,KEGG_DRUG,5278,NaN
1,1214713,9745,IUPHAR_LIGAND_ID,5271,NaN


In [4]:
syn = pd.read_csv('../data/drugcentral_syn_06212018.csv')
syn.rename(columns={'id': 'struct_id'}, inplace=True)
syn.head(2)

,syn_id,struct_id,name,preferred_name,parent_id,lname
0,22490,5253.0,RPX-7009,NaN,NaN,rpx-7009
1,22493,5003.0,insulin (human),NaN,NaN,insulin (human)


In [5]:
pref = syn.query('preferred_name == 1').reset_index(drop=True)
pref = pref.dropna(subset=['struct_id'])
pref['struct_id'] = pref['struct_id'].astype('int64')
pref.head(2)

,syn_id,struct_id,name,preferred_name,parent_id,lname
0,22368,5253,vaborbactam,1.0,NaN,vaborbactam
1,22371,5254,levornidazole,1.0,NaN,levornidazole


In [6]:
struct_id_to_name = pref.set_index('struct_id')['name'].to_dict()
rels['c_name'] = rels['struct_id'].map(lambda i: struct_id_to_name.get(i, float('nan')))

In [7]:
rels.shape[0] == rels['c_name'].count()

True

## 2. Map the Compounds in Semmed DB to MeSH

Although we will be mapping all UMLS CUIs (that can be mapped) to MeSH, after the inital map, we will start by taking a closer look at the compounds.  Because there are multiple sources of X-refs for both Compounds and Diseases, these special Metanodes will be a bit more complicated than a simple direct map.

Starting with a direct Map from UMLS to MeSH will combine a lot of the Compound nodes, reducing the number of total unique compounds.

In [8]:
nodes = gt.remove_colons(pd.read_csv('../data/nodes_VER31_R.csv'))
umls_to_mesh = pickle.load(open('../data/UMLS-CUI_to_MeSH-Descripctor.pkl', 'rb'))

In [9]:
umls_to_mesh_1t1 = {k: v[0] for k, v in umls_to_mesh.items() if len(v) == 1}

In [10]:
nodes['mesh_id'] = nodes['id'].map(lambda c: umls_to_mesh_1t1.get(c, float('nan')))

In [11]:
drugs = nodes.query('label == "Chemicals & Drugs"').copy()

In [12]:
print('{:.3%} of Drug IDs mapped via MeSH:'.format(drugs['mesh_id'].count() / drugs.shape[0]))
print('{:,} of {:,} Mapped to {:,} Unique MSH ids'.format(drugs['mesh_id'].count(), drugs.shape[0], drugs['mesh_id'].nunique()))

num_drugs = drugs['id'].nunique()
msh_compress_drugs = drugs['mesh_id'].fillna(drugs['id']).nunique()

print('{:.3%} Reduction in Drugs by using MSH synonmyms {:,} --> {:,}'.format((num_drugs - msh_compress_drugs)/num_drugs, num_drugs, msh_compress_drugs))

86.783% of Drug IDs mapped via MeSH:
82,236 of 94,761 Mapped to 66,939 Unique MSH ids
16.143% Reduction in Drugs by using MSH synonmyms 94,761 --> 79,464


## 3. Use UMLS MeSH mappings and Mappings from DrugCentral to ensure Maximum overlap

DrugCentral also has it's own internal identifiers for compounds as well as mappings from both their internal id to UMLS and MeSH.  

If we treat these mappings all as edges in a network, and use a Subnet finding algorthim, each subnet will essentially be a unique chemical structure, with the nodes of that subnet representing all of the different identifiers that map to that structure.

In [13]:
dc_maps = dc_ids.query('id_type in {}'.format(["MESH_DESCRIPTOR_UI", "MESH_SUPPLEMENTAL_RECORD_UI" , "UMLSCUI"]))

drug_adj_list = defaultdict(set)

for row in tqdm(dc_maps.itertuples(), total=len(dc_maps)):
    drug_adj_list[row.struct_id].add(row.identifier)
    drug_adj_list[row.identifier].add(row.struct_id)

100%|██████████| 7015/7015 [00:00<00:00, 27696.11it/s]


In [14]:
umls_keys = list(chain(*[[k]*len(v) for k, v in umls_to_mesh.items()]))
mesh_vals = list(chain(*[v for v in umls_to_mesh.values()]))

In [15]:
umls_to_mesh_df = pd.DataFrame({'umls': umls_keys, 'mesh': mesh_vals})
drug_ids = drugs['id'].unique()
umls_to_mesh_drugs = umls_to_mesh_df.query('umls in @drug_ids')

In [16]:
umls_set = set(drugs['id']) | set(dc_maps.query('id_type == "UMLSCUI"'))
mesh_set = set(mesh_vals) | set(dc_maps.query('id_type in {}'.format(["MESH_DESCRIPTOR_UI", "MESH_SUPPLEMENTAL_RECORD_UI"]))['identifier'])

len(umls_set & mesh_set) == 0

True

In [17]:
for row in umls_to_mesh_drugs.itertuples():
    drug_adj_list[row.umls].add(row.mesh)
    drug_adj_list[row.mesh].add(row.umls)

In [18]:
def get_subnets(adj_list):

    all_identifiers = set(adj_list.keys())

    subnets = defaultdict(set)
    visited = set()

    for cui in tqdm(all_identifiers):
        if cui not in visited:
            visited.add(cui)
            q = Queue()
            q.put(cui)

            while not q.empty():
                cur = q.get()
                visited.add(cur)

                for neighbour in adj_list[cur]:
                    subnets[cui].add(neighbour)
                    if neighbour not in visited:
                        q.put(neighbour)
                        visited.add(neighbour)

    return subnets

In [19]:
subnets = get_subnets(drug_adj_list)

100%|██████████| 154161/154161 [00:03<00:00, 47027.46it/s]


In [20]:
len(subnets)

67207

Find a label for each group.  
Will choose based on number of umls items that can be mapped to a single MeSH term (more == higher priority).

In [21]:
mesh_counts = umls_keys + mesh_vals + list(dc_maps['identifier']) + list(dc_maps['struct_id'].unique())
mesh_counts = Counter(mesh_counts)

rekeyed_subnets = dict()

for v in subnets.values():
    sort_sub = sorted(list(v), key=lambda k: (mesh_counts[k], k in mesh_set, k in umls_set), reverse=True)
    new_key = sort_sub[0]
    rekeyed_subnets[new_key] = v

In [22]:
# Final map is just inverse of the subnets dict
final_drug_map = dict()

for k, v in rekeyed_subnets.items():
    for val in v:
        final_drug_map[val] = k

In [23]:
len(final_drug_map)

154161

In [24]:
pickle.dump(final_drug_map, open('../data/drug_merge_map.pkl', 'wb'))

## 4. Map all the compounds and check

Do a final mapping of the compound IDs for merging, and spot check a few results

In [25]:
# Some items won't necessarily be mappable, so use original ID
drugs['new_id'] = drugs['id'].map(lambda i: final_drug_map.get(i, i))

In [26]:
# Map the Gold Standard indications as well
rels['compound_new_id'] = rels['struct_id'].map(lambda i: final_drug_map.get(i, i))

In [27]:
drugs['id_source'] = drugs['new_id'].map(lambda x: 'MeSH' if x in mesh_set else 'UMLS')

In [28]:
drugs.head(2)

,id,name,label,mesh_id,new_id,id_source
16644,C0113168,desciclovir,Chemicals & Drugs,C041468,C041468,MeSH
16645,C0140749,Ro 14-4767-002,Chemicals & Drugs,C038974,C038974,MeSH


In [29]:
print('{:.3%} Reduction in Drugs {:,} --> {:,}'.format(
    (drugs.shape[0] - drugs['new_id'].nunique())/drugs.shape[0], drugs.shape[0], drugs['new_id'].nunique()))

16.397% Reduction in Drugs 94,761 --> 79,223


In [30]:
inds = rels.query('relationship_name == "indication"')

drug_ids_semmed = set(drugs['new_id'])
drugs_in_inds = set(inds['compound_new_id'].dropna())

num_ind_in_semmed = len(drugs_in_inds & drug_ids_semmed)

print('{:.3%} of Drugs in DC Indications mapped: {:,} out of {:,}'.format(
    (num_ind_in_semmed / len(drugs_in_inds)), num_ind_in_semmed, len(drugs_in_inds)))

ind_semmed_comp = inds.query('compound_new_id in @drug_ids_semmed').shape[0]

print('{:.3%} of Indications have mappable Drug: {:,} out of {:,}'.format(
    (ind_semmed_comp / len(inds)), ind_semmed_comp, len(inds)))


85.423% of Drugs in DC Indications mapped: 2,010 out of 2,353
95.209% of Indications have mappable Drug: 10,414 out of 10,938


Looks at the  Mesh IDs that mapped to the greatest number of CUIs and see if the mappings make sense...

In [31]:
mesh_counts.most_common(3)

[('D014147', 56), ('D024505', 55), ('D000111', 47)]

In [32]:
to_q = mesh_counts.most_common(3)[0][0]

drugs.query('new_id == @to_q')

,id,name,label,mesh_id,new_id,id_source
26009,C0592292,Zydol,Chemicals & Drugs,D014147,D014147,MeSH
28681,C0724054,Ultram,Chemicals & Drugs,D014147,D014147,MeSH
36318,C2350089,clorhidrato de tramadol (producto),Chemicals & Drugs,D014147,D014147,MeSH
68080,C0728935,K-315,Chemicals & Drugs,D014147,D014147,MeSH
100994,C0040610,Tramadol,Chemicals & Drugs,D014147,D014147,MeSH
110191,C0040611,Tramal,Chemicals & Drugs,D014147,D014147,MeSH


In [33]:
to_q = mesh_counts.most_common(3)[1][0]

drugs.query('new_id == @to_q')

,id,name,label,mesh_id,new_id,id_source
27929,C0115296,E-ferol,Chemicals & Drugs,D024505,D024505,MeSH
94056,C0591450,Ephynal,Chemicals & Drugs,D024505,D024505,MeSH
107206,C0087096,Tocopherols,Chemicals & Drugs,D024505,D024505,MeSH


In [34]:
to_q = mesh_counts.most_common(3)[2][0]

drugs.query('new_id == @to_q')

,id,name,label,mesh_id,new_id,id_source
20046,C1449437,Jenacystein,Chemicals & Drugs,D000111,D000111,MeSH
27415,C0591477,Fabrol,Chemicals & Drugs,D000111,D000111,MeSH
52322,C1449416,Alveolex,Chemicals & Drugs,D000111,D000111,MeSH
54835,C1449405,Siccoral,Chemicals & Drugs,D000111,D000111,MeSH
83284,C0699252,Mucomyst,Chemicals & Drugs,D000111,D000111,MeSH
101647,C0699253,Mucosolvin,Chemicals & Drugs,D000111,D000111,MeSH
105279,C0699251,Fluimucil,Chemicals & Drugs,D000111,D000111,MeSH
106225,C0001047,Acetylcysteine,Chemicals & Drugs,D000111,D000111,MeSH


These all look pretty good.  All of the names for `D000111` are listed uner the aliases on the [Acetylcysteine MeSH page](https://meshb.nlm.nih.gov/record/ui?ui=D000111)

Now let's look at a few compounds that may have a new MeSH ID distinct from their original, thanks to incorporating the DrugCentral X-refs

In [37]:
new_id_not_mesh = drugs.dropna(subset=['mesh_id']).query('new_id != mesh_id')
print(len(new_id_not_mesh))
new_id_not_mesh.head(10)

314


,id,name,label,mesh_id,new_id,id_source
16919,C0054425,caerulein diethylamine,Chemicals & Drugs,C032637,D002108,MeSH
16999,C0383405,Gonal F,Chemicals & Drugs,C571801,D050477,MeSH
17185,C1257665,Aclaplastin,Chemicals & Drugs,D015250,C011157,MeSH
17647,C1570410,dicaffeoylquinic acid,Chemicals & Drugs,C472707,C100257,MeSH
17716,C0809900,MK-412A,Chemicals & Drugs,D002710,C010882,MeSH
17827,C0077209,trimetrexate glucuronate,Chemicals & Drugs,C056321,D016597,MeSH
18149,C0717550,candesartan,Chemicals & Drugs,C081643,C077793,MeSH
18872,C0546173,Calcium ascorbate,Chemicals & Drugs,C069207,D001205,MeSH
19803,C0591649,Innohep,Chemicals & Drugs,C081247,D017985,MeSH
20240,C0141227,S-9780,Chemicals & Drugs,C053500,D020913,MeSH


In [38]:
diff_new_ids = new_id_not_mesh.query('id_source == "MeSH"')['new_id'].values

In [39]:
diff_new_ids[:5]

array(['D002108', 'D050477', 'C011157', 'C100257', 'C010882'],
      dtype=object)

In [40]:
inds.query('compound_new_id == "D002108"')

,id,struct_id,concept_id,relationship_name,concept_name,umls_cui,snomed_full_name,cui_semantic_type,snomed_conceptid,c_name,compound_new_id
630,133039,579,21001931,indication,Diagnostic aid,C0358514,Diagnostic aid,T130,2949005.0,ceruletide,D002108


## 5. Diseases:

With diseases we will do a little more in terms of the mapping.  Beacause diseases in both UMLS and MeSH, we will incorporate some of the mappings from Disease Ontology Slim to try and get general diseases.  The workflow will be as folows:

1. Map nodes to Mesh
2. Map ind CUI and/or SNOMED terms from DrugCentral to Mesh
4. Incorporate DO Slim mappings
3. Find overlap between these soruces


In [41]:
diseases = nodes.query('label == "Disorders"').copy()

In [42]:
len(diseases)

40798

In [43]:
dis_numbers = diseases.groupby('mesh_id').apply(len).sort_values(ascending=False)

In [44]:
param = dis_numbers[:10].index.tolist()
diseases.query('mesh_id in @param').sort_values('mesh_id')

,id,name,label,mesh_id
131524,C0004114,Astrocytoma,Disorders,D001254
148138,C0334580,Protoplasmic astrocytoma,Disorders,D001254
133067,C0338070,Childhood Cerebral Astrocytoma,Disorders,D001254
147858,C0334583,Pilocytic Astrocytoma,Disorders,D001254
151600,C0205768,Subependymal Giant Cell Astrocytoma,Disorders,D001254
125768,C0750935,Cerebral Astrocytoma,Disorders,D001254
158858,C0334581,Gemistocytic astrocytoma,Disorders,D001254
157596,C0280783,Juvenile Pilocytic Astrocytoma,Disorders,D001254
159201,C0750936,Intracranial Astrocytoma,Disorders,D001254
148012,C0547065,Mixed oligoastrocytoma,Disorders,D001254


In [45]:
conso = load_umls.open_mrconso()

/opt/mmayers/software/anaconda3/envs/integrate/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2862: DtypeWarning: Columns (9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
conso.head(2)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0


In [47]:
snomed_xrefs = conso.query("SAB == 'SNOMEDCT_US'").dropna(subset=['CUI', 'SCUI'])

In [48]:
snomed_xrefs.head(2)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
20,C0000039,ENG,S,L0012507,PF,S0033298,N,A22817493,166113012.0,102735002,NaN,SNOMEDCT_US,OAP,102735002,Dipalmitoylphosphatidylcholine,9,O,256.0
29,C0000039,ENG,S,L3000054,PF,S3260062,Y,A22880204,544223010.0,102735002,NaN,SNOMEDCT_US,OAF,102735002,Dipalmitoylphosphatidylcholine (substance),9,O,NaN


In [49]:
dis_adj_list = defaultdict(set)

disease_ids = set(diseases['id'].unique())

umls_to_mesh_dis = umls_to_mesh_df.query('umls in @disease_ids')

for row in umls_to_mesh_dis.itertuples():
    dis_adj_list[row.umls].add(row.mesh)
    dis_adj_list[row.mesh].add(row.umls)

In [50]:
# Convert the snomed concept ids to string since they're strings the adj_list
rels['snomed_conceptid'] = rels['snomed_conceptid'].map(lambda i: str(int(i)) if not pd.isnull(i) else i)

sub_rels = rels.dropna(subset=['snomed_conceptid', 'umls_cui'])

for row in sub_rels.itertuples():
    dis_adj_list[row.umls_cui].add(row.snomed_conceptid)
    dis_adj_list[row.snomed_conceptid].add(row.umls_cui)
    # Make sure to get mesh to CUI maps for the new cuis picked up via drugcentral
    if row.umls_cui in umls_to_mesh_1t1:
        dis_adj_list[umls_to_mesh_1t1[row.umls_cui]].add(row.umls_cui)
        dis_adj_list[row.umls_cui].add(umls_to_mesh_1t1[row.umls_cui])

In [51]:
ind_snomed = set(rels['snomed_conceptid'])
dis_umls = set(rels['umls_cui']) | disease_ids

dis_snomed_xrefs = snomed_xrefs.query('CUI in @dis_umls or SCUI in @ind_snomed')

print(len(dis_snomed_xrefs))

for row in tqdm(dis_snomed_xrefs.itertuples(), total=len(dis_snomed_xrefs)):
    dis_adj_list[row.CUI].add(row.SCUI)
    dis_adj_list[row.SCUI].add(row.CUI)
    # Make sure to get mesh to CUI maps for the new cuis picked up via drugcentral
    if row.CUI in umls_to_mesh_1t1:
        dis_adj_list[umls_to_mesh_1t1[row.CUI]].add(row.CUI)
        dis_adj_list[row.CUI].add(umls_to_mesh_1t1[row.CUI])

 11%|█         | 17658/163590 [00:00<00:00, 176576.38it/s]

163590


100%|██████████| 163590/163590 [00:00<00:00, 270281.76it/s]


### DO Slim Integration

The following disease-ontology files were generated from a [fork of Daniel Himmelstein's work generating the Disease Ontology Slim](https://github.com/mmayers12/disease-ontology).  The only major differnece between Daniel's Release and this version is that I have added in the Disease Ontology terms from their 'Rare Slim' list to attempt to get some coverage of Rare Monogetic Diseases.  These can be another way to consolidate diesease into more general types

First we'll need a DOID to UMLS_CUI map, WikiData can provide a quick and dirty map

In [52]:
from wikidataintegrator import wdi_core

query_text = """
select ?doid ?umlscui

WHERE
{
    ?s wdt:P699 ?doid .
    ?s wdt:P2892 ?umlscui .
}
"""

result = wdi_core.WDItemEngine.execute_sparql_query(query_text, as_dataframe=True)
result.to_csv('../data/doid-to-umls.csv', index=False)
doid_to_umls = result.set_index('doid')['umlscui'].to_dict()

In [53]:
slim_xref = pd.read_table('../../disease-ontology/data/xrefs-prop-slim.tsv')
do_slim = pd.read_table('../../disease-ontology/data/slim-terms-prop.tsv')

In [54]:
slim_xref.head(2)

,doid_code,doid_name,resource,resource_id
0,DOID:2531,hematologic cancer,CSP,2004-1600
1,DOID:2531,hematologic cancer,CSP,2004-1803


In [55]:
slim_xref['resource'].value_counts()

SNOMEDCT_US_2016_03_01    3275
OMIM                      2472
UMLS                      2420
NCI                       1984
ICD10                      841
MESH                       805
ORDO                       685
ICD9                       561
EFO                        116
KEGG                        23
HP                          14
CSP                         12
MedDRA                       5
SNOMEDCT                     2
SNOMEDCT_US_2015_03_01       1
NDFRT                        1
GARD                         1
Name: resource, dtype: int64

In [56]:
resources = ['SNOMEDCT_US_2016_03_01', 'UMLS', 'MESH', 'SNOMEDCT', 'SNOMEDCT_US_2015_03_01']
useful_xref = slim_xref.query('resource in @resources')

In [57]:
for row in useful_xref.itertuples():
    dis_adj_list[row.doid_code].add(row.resource_id)
    dis_adj_list[row.resource_id].add(row.doid_code)
    if row.resource == "UMLS" and row.resource_id in umls_to_mesh_1t1:
        dis_adj_list[umls_to_mesh_1t1[row.resource_id]].add(row.resource_id)
        dis_adj_list[row.resource_id].add(umls_to_mesh_1t1[row.resource_id])

In [58]:
do_slim.head(2)

,slim_id,slim_name,subsumed_id,subsumed_name,min_distance
0,DOID:0050144,Kartagener syndrome,DOID:0050144,Kartagener syndrome,0
1,DOID:0050156,idiopathic pulmonary fibrosis,DOID:0050156,idiopathic pulmonary fibrosis,0


In [59]:
do_slim['cui'] = do_slim['subsumed_id'].map(lambda d: doid_to_umls.get(d, float('nan')))
do_slim_d = do_slim.dropna(subset=['cui'])

In [60]:
for row in do_slim_d.itertuples():
    dis_adj_list[row.subsumed_id].add(row.cui)
    dis_adj_list[row.cui].add(row.subsumed_id)
    if row.cui in umls_to_mesh_1t1:
        dis_adj_list[umls_to_mesh_1t1[row.cui]].add(row.cui)
        dis_adj_list[row.cui].add(umls_to_mesh_1t1[row.cui])

## 6. Make the final map for Diseases and Map them

In [61]:
dis_subnets = get_subnets(dis_adj_list)

100%|██████████| 91444/91444 [00:01<00:00, 53582.32it/s]


In [62]:
len(dis_subnets)

25476

In [63]:
umls_set = set(diseases['id'].dropna()) | set(rels['umls_cui'].dropna())
umls_to_val = {u: 9999999-int(u[1:]) for u in umls_set}

mesh_counts = umls_keys + mesh_vals + list(rels['umls_cui'].map(lambda c: umls_to_mesh_1t1.get(c, c))) 
mesh_counts = Counter(mesh_counts)

rekeyed_dis_subnets = dict()

for v in dis_subnets.values():
    # First take ones in the mesh, then by the highest number of things it consolidated
    # Then take the lowest numbered UMLS ID...
    sort_sub = sorted(list(v), key=lambda k: (k in mesh_set, mesh_counts[k], k in umls_set, umls_to_val.get(k, 0)), reverse=True)
    new_key = sort_sub[0]
    rekeyed_dis_subnets[new_key] = v

In [64]:
'C565169' in mesh_vals

True

In [65]:
# Final map is just inverse of the subnets dict
final_dis_map = dict()

for k, v in rekeyed_dis_subnets.items():
    for val in v:
        final_dis_map[val] = k

In [66]:
diseases['new_id'] = diseases['id'].map(lambda i: final_dis_map.get(i, i))

In [67]:
# See how many instances of diseases mapped to 1 mesh ID had their ID changed through
# SNOMED and DO-SLIM consolidation
print('{} original CUIs'.format(diseases.dropna(subset=['mesh_id']).query('mesh_id != new_id')['id'].nunique()))
print('Mapped to {} MeSH IDs'.format(diseases.dropna(subset=['mesh_id']).query('mesh_id != new_id')['mesh_id'].nunique()))
print('Consolidated to {} unique entities'.format(diseases.dropna(subset=['mesh_id']).query('mesh_id != new_id')['new_id'].nunique()))

1992 original CUIs
Mapped to 1190 MeSH IDs
Consolidated to 451 unique entities


In [68]:
def dis_source_map(x):
    if x in mesh_set:
        return 'MeSH'
    elif x in umls_set:
        return 'UMLS'
    else:
        # Just in case there's a problem...
        return 'Uh-Oh'

diseases['id_source'] = diseases['new_id'].map(lambda x: dis_source_map(x))

In [69]:
diseases['id_source'].value_counts()

UMLS    30190
MeSH    10608
Name: id_source, dtype: int64

In [70]:
pickle.dump(final_dis_map, open('../data/disease_merge_map.pkl', 'wb'))

In [71]:
print('{:.3%} Reduction in Diseases {:,} --> {:,}'.format(
    (diseases.shape[0] - diseases['new_id'].nunique())/diseases.shape[0], diseases.shape[0], diseases['new_id'].nunique()))

15.866% Reduction in Diseases 40,798 --> 34,325


In [72]:
rels['disease_new_id'] = rels['umls_cui'].map(lambda c: final_dis_map.get(c, c))
print(rels['disease_new_id'].count())
bad_idx = rels[rels['disease_new_id'].isnull()].index

rels.loc[bad_idx, 'disease_new_id'] = rels.loc[bad_idx, 'snomed_conceptid'].map(lambda c: final_dis_map.get(c, float('nan')))

37005


In [73]:
inds = rels.query('relationship_name == "indication"')

disease_ids_semmed = set(diseases['new_id'])
diseases_in_inds = set(inds['disease_new_id'].dropna())

num_ind_in_semmed = len(diseases_in_inds & disease_ids_semmed)

print('{:.3%} of diseases in DC Indications mapped: {:,} out of {:,}'.format(
    (num_ind_in_semmed / len(diseases_in_inds)), num_ind_in_semmed, len(diseases_in_inds)))

ind_semmed_comp = inds.query('disease_new_id in @disease_ids_semmed').shape[0]

print('{:.3%} of Indications have mappable disease: {:,} out of {:,}'.format(
    (ind_semmed_comp / len(inds)), ind_semmed_comp, len(inds)))

80.819% of diseases in DC Indications mapped: 868 out of 1,074
69.062% of Indications have mappable disease: 7,554 out of 10,938


In [74]:
inds_dd = inds.drop_duplicates(subset=['compound_new_id', 'disease_new_id'])

new_cids = set(drugs['new_id'].unique())
new_dids = set(diseases['new_id'].unique())

inds_in_semmed = inds_dd.query('compound_new_id in @new_cids and disease_new_id in @new_dids')
print('{:.3%} of indications now have both compound and disease mappable {:,} out of {:,}'.format(
    len(inds_in_semmed) / len(inds_dd), len(inds_in_semmed), len(inds_dd)))

76.181% of indications now have both compound and disease mappable 6,307 out of 8,279


### Add in Dates for Indications

Since the Indications are pretty much fully mapped to the network and ready to go as a Gold Standard for machine learning, we will map approval date information to the compounds now, so it's available for future analyses.

In [75]:
app = pd.read_csv('../data/drugcentral_approvals_06212018.csv')
app.head()

,id,struct_id,approval,type,applicant,orphan
0,3618,322,2017-08-29,FDA,CHEMO RESEARCH SL,t
1,3619,5253,2017-08-29,FDA,REMPEX PHARMS INC,NaN
2,3620,5254,2009-08-13,China Food and Drug Administration (CFDA),Sanhome,NaN
3,3622,5255,2014-02-24,China Food and Drug Administration (CFDA),Hansoh,NaN
4,3623,5256,2017-09-14,FDA,BAYER HEALTHCARE PHARMS,t


In [76]:
app = app.rename(columns={'approval': 'approval_date'})

app = (app.dropna(subset=['approval_date']) # Remove NaN values
          .sort_values('approval_date')     # Put the earliest approval_date first
          .groupby('struct_id')        # Group by the compound's id
          .first()                     # And select the first instance of that id
          .reset_index())              # Return struct_id to a column from the index

In [77]:
rels = pd.merge(rels, app[['struct_id', 'approval_date']], how='left', on='struct_id')
rels.head(2)

,id,struct_id,concept_id,relationship_name,concept_name,umls_cui,snomed_full_name,cui_semantic_type,snomed_conceptid,c_name,compound_new_id,disease_new_id,approval_date
0,173432,965,40249340,indication,Malignant tumor of breast,C0006142,Malignant tumor of breast,T191,254837009,drostanolone propionate,C007561,D001943,NaN
1,173433,318,21000716,indication,Gout,C0018099,Gout,T047,90560007,benzbromarone,D001553,D006073,NaN


In [78]:
idx = rels[~rels['approval_date'].isnull()].index
rels.loc[idx, 'approval_year'] = rels.loc[idx, 'approval_date'].map(lambda s: s.split('-')[0])

In [79]:
rels.head(2)

,id,struct_id,concept_id,relationship_name,concept_name,umls_cui,snomed_full_name,cui_semantic_type,snomed_conceptid,c_name,compound_new_id,disease_new_id,approval_date,approval_year
0,173432,965,40249340,indication,Malignant tumor of breast,C0006142,Malignant tumor of breast,T191,254837009,drostanolone propionate,C007561,D001943,NaN,NaN
1,173433,318,21000716,indication,Gout,C0018099,Gout,T047,90560007,benzbromarone,D001553,D006073,NaN,NaN


## 7. Rebuild the Nodes

The node CSV will now be rebuilt with all the new ID mappings and corresponding concept names

In [80]:
all_umls = set(nodes['id'])

In [81]:
umls_set = set(nodes['id']) | set(dc_maps.query('id_type == "UMLSCUI"')) |  set(rels['umls_cui'])

In [82]:
def get_source(cid):
    if cid in mesh_set:
        return 'MeSH'
    elif cid in umls_set:
        return 'UMLS'
    else:
        return 'problem...'

In [83]:
pickle.dump(umls_set, open('../data/umls_id_set.pkl', 'wb'))
pickle.dump(mesh_set, open('../data/mesh_id_set.pkl', 'wb'))

In [84]:
new_nodes = nodes.query('label not in {}'.format(['Chemicals & Drugs', 'Disorders'])).copy()

new_nodes['new_id'] = new_nodes['mesh_id'].fillna(new_nodes['id'])
new_nodes['id_source'] = new_nodes['new_id'].apply(lambda c: get_source(c))
new_nodes['id_source'].value_counts()

UMLS    104999
MeSH     16685
Name: id_source, dtype: int64

In [85]:
drug_dis = pd.concat([drugs, diseases])

In [86]:
curr_map = drug_dis.set_index('id')['new_id'].to_dict()

In [87]:
idx = drug_dis.groupby('new_id')['label'].nunique() > 1
problems = idx[idx].index.values
print(len(problems))

4


In [88]:
remap = dict()
grpd = drug_dis.query('new_id in @problems').groupby('new_id')

for grp, df in grpd:
    for labels in df['label'].unique():
        curr_label = df.query('label == @labels')['id'].values
        
        # Keep the MeSH Map for the New ID if its a Drug
        if labels == 'Chemcials & Drugs':
            for c in curr_label:
                remap[c] = grp
                
        # Use a random Disease CUI if its a Disease
        else:
            new_cui = curr_label[0]
            for c in curr_label:
                remap[c] = new_cui

drug_dis['new_id'] = drug_dis['id'].map(lambda i: remap.get(i, curr_map[i]))

#### Go back and Fix the Indications

We just changed 4 Diseases back to CUIs, so must ensure those don't affect the earlier mappings to indicaitons

In [89]:
if rels.query('disease_new_id in @problems').shape[0] > 0:
    print('This is a problem')
else:
    print('This is a non-issue so no need to fix anything')

This is a non-issue so no need to fix anything


In [90]:
new_nodes = pd.concat([new_nodes, drug_dis])
new_nodes = new_nodes.sort_values('label')

idx = new_nodes.groupby('new_id')['label'].nunique() > 1
problems = idx[idx].index.values
print(len(problems))

494


In [91]:
new_nodes.query('new_id in {}'.format(problems.tolist())).sort_values('new_id').head(10)

,id,name,label,mesh_id,new_id,id_source
63178,C0124261,ivalon powder,Chemicals & Drugs,C026699,C026699,MeSH
105339,C0377284,ivalon,Chemicals & Drugs,C026699,C026699,MeSH
117014,C0064121,ivalon sponge,Devices,C026699,C026699,MeSH
119630,C0137919,polyvinyl alcohol foam,Devices,C026699,C026699,MeSH
20010,C0060524,fluorescein dilaurate,Chemicals & Drugs,C034833,C034833,MeSH
248027,C0430194,Pancreolauryl test,Procedures,C034833,C034833,MeSH
54883,C0912748,TEC protocol,Chemicals & Drugs,C405323,C405323,MeSH
247002,C0912750,TEC regimen,Procedures,C405323,C405323,MeSH
28130,C0288896,HRP 102,Chemicals & Drugs,C418365,C418365,MeSH
30736,C0256136,Mesigyna,Chemicals & Drugs,C418365,C418365,MeSH


### Fix other node-type conflicts

Since the UMLS to MeSH map has no regard for semmantic type of the node, some concepts may have been condensed across semmantic types.

All the Drug and Disease overlaps should be solved, so now move onto other nodetype conflicts.

Conflicts will be solved in this manner:

1. If one of the types is a Drug or a Disease, that one gets the MeSH ID
2. If no Drug or Disease, the one that has the largest number of nodes will recieve the MeSH ID
3. Remaining Nodetypes will be separated and assume the CUI of the node with the highest degree of connection in the network


Take, for example, `Ivalon`.  It has 4 original CUIs that mapped to the same MeSH ID.  Two of which carried the semmantic type `Chemicals & Drugs` and two `Devices`.  The mesh_id will be kept for the `Chemicals & Drugs` version of the Nodes, which will be merged.  The `Devices` versions of the nodes will be merged, whichever CUI has the most has the greatest number of edges will be the CUI used for this merged node.  

`Chemicals & Drugs` and `Disorders` will always take the meshID before other semmantic types.  Otherwise, the MeSH id will be assignged to the semanitc type had the most CUIs merged into 1 node. The other semmatic types will again have the CUI selected based on edge count.

In [92]:
edges = gt.remove_colons(pd.read_csv('../data/edges_VER31_R.csv', converters={'pmids':eval}))

In [93]:
cui_counts = edges['start_id'].value_counts().add(edges['end_id'].value_counts(), fill_value=0).to_dict()

In [94]:
# For now, just return conflicting nodes to thier old semmantic type
grpd = new_nodes.query('new_id in @problems').groupby('new_id')
remap = dict()

for msh_id, df in tqdm(grpd, total=len(grpd)):

    # Get all the labels and counts for those labels
    labels = df['label'].unique().tolist()
    counts = df['label'].value_counts().to_dict()
    
    # Sort the by the Number of different nodes mapped to that label
    labels = sorted(labels, key=lambda l: counts[l], reverse=True)
    
    # Chemicals and Drugs and Diseases have higher priorities in the context of machine learning
    # So any item that could be either of those types will be set to them automatically.
    drug_or_dis = False
    
    # Select the Chemicals & Drugs nodes to have the MeSH ID if possible
    if 'Chemicals & Drugs' in labels:
        labels.remove('Chemicals & Drugs')
        curr_label = df.query('label == "Chemicals & Drugs"')['id'].values
        drug_or_dis = True
        for c in curr_label:
            remap[c] = msh_id    
    
    # Otherwise, elect the Disorders nodes to have the MeSH ID if possible
    elif 'Disorders' in labels:
        labels.remove('Disorders')
        curr_label = df.query('label == "Disorders"')['id'].values
        drug_or_dis = True
        for c in curr_label:
            remap[c] = msh_id    
    
    # Finally assign a merged CUI based on edge counts
    for i, label in enumerate(labels):
        curr_label = df.query('label == @label')['id'].values
        
        # Give highest counts of nodes the MeSH ID, if not already assigned to a Drug or Disease
        if i == 0 and not drug_or_dis:
            new_cui = msh_id
        else:
            # For types that won't get a MeSH ID, 
            # get the CUI that has largest number of instances in the edges
            new_cui = sorted(curr_label, key=lambda v: cui_counts.get(v, 0), reverse=True)[0]
        for c in curr_label:
            remap[c] = new_cui

100%|██████████| 494/494 [00:02<00:00, 199.48it/s]


In [95]:
# Perform the new Mapping
curr_map = new_nodes.set_index('id')['new_id'].to_dict()
new_nodes['new_id'] = nodes['id'].map(lambda i: remap.get(i, curr_map[i]))


# Ensure there are now no problems
idx = new_nodes.groupby('new_id')['label'].nunique() > 1
problems = idx[idx].index.values
print(len(problems))

0


In [96]:
num_old_ids = new_nodes['id'].nunique()
num_new_ids = new_nodes['new_id'].nunique()

print('{:.3%} reduction in the number of NODES\n{:,} --> {:,}'.format((num_old_ids-num_new_ids)/num_old_ids, num_old_ids, num_new_ids))

10.333% reduction in the number of NODES
257,243 --> 230,661


In [97]:
new_nodes['id_source'] = new_nodes['new_id'].apply(lambda c: get_source(c))
new_nodes['id_source'].value_counts()

UMLS    148296
MeSH    108947
Name: id_source, dtype: int64

In [98]:
cui_to_name = nodes.set_index('id')['name'].to_dict()
cui_to_name = {**cui_to_name, **rels.set_index('umls_cui')['concept_name'].to_dict()}
cui_to_name = {**cui_to_name, **rels.set_index('compound_new_id')['c_name'].to_dict()}

msh_to_name = pickle.load(open('../data/MeSH_DescUID_to_Name.pkl', 'rb'))
# The mappings from UMLS are less reliable, so use the ones that came from MeSH itself first
msh_to_name = {**pickle.load(open('../data/MeSH_id_to_name_via_UMLS.pkl', 'rb')), **msh_to_name}

id_to_name = {**cui_to_name, **msh_to_name}

In [99]:
# All new IDs should have a mapped name
set(new_nodes['new_id']).issubset(set(id_to_name.keys()))

True

In [100]:
new_nodes['name'] = new_nodes['new_id'].map(lambda i: id_to_name[i])

In [101]:
pickle.dump(id_to_name, open('../data/all_ids_to_names.pkl', 'wb'))

In [102]:
final_node_map = new_nodes.set_index('id')['new_id'].to_dict()

## 8. Map all the edges

Now that we have a finalized original to new ID map, we can straight map all the ids in the edges file.

If any edges are now duplicated, PMIDs in support for those edges will be merged into a set.

In [103]:
edges['start_id'] = edges['start_id'].map(lambda c: final_node_map[c])
edges['end_id'] = edges['end_id'].map(lambda c: final_node_map[c])

In [104]:
%%time

num_before = len(edges)

# Some edges now duplicated, de-duplicate and combine pmids
grpd = edges.groupby(['start_id', 'end_id', 'type'])
edges = grpd['pmids'].apply(lambda Series: set.union(*Series.values)).reset_index()

# re-count the pmid numbers
edges['n_pmids'] = edges['pmids'].apply(len)

num_after = len(edges)

CPU times: user 32min 23s, sys: 38.8 s, total: 33min 2s
Wall time: 33min 2s


In [105]:
print('{:,} Edges before node consolidation'.format(num_before))
print('{:,} Edges after node consolidation'.format(num_after))
print('A {:.3%} reduction in edges'.format((num_before - num_after) / num_before))

19,555,814 Edges before node consolidation
18,145,463 Edges after node consolidation
A 7.212% reduction in edges


### Save the files for the network

In [106]:
# Get rid of the old ids in the nodes
new_nodes.drop('id', axis=1, inplace=True)
new_nodes = new_nodes.rename(columns={'new_id': 'id'})[['id', 'name', 'label', 'id_source']]
new_nodes = new_nodes.drop_duplicates(subset='id')

# Sort values before writing to disk
new_nodes = new_nodes.sort_values('label')
edges = edges.sort_values('type')

# Add in colons required by neo4j
new_nodes = gt.add_colons(new_nodes)
edges = gt.add_colons(edges)

new_nodes.to_csv('../data/nodes_VER31_R_nodes_consolidated.csv', index=False)
edges.to_csv('../data/edges_VER31_R_nodes_consolidated.csv', index=False)
pickle.dump(final_node_map, open('../data/node_id_merge_map.pkl', 'wb'))

### Save the relationship files for a Machine Learning Gold Standard

In [107]:
rels.head(2)

,id,struct_id,concept_id,relationship_name,concept_name,umls_cui,snomed_full_name,cui_semantic_type,snomed_conceptid,c_name,compound_new_id,disease_new_id,approval_date,approval_year
0,173432,965,40249340,indication,Malignant tumor of breast,C0006142,Malignant tumor of breast,T191,254837009,drostanolone propionate,C007561,D001943,NaN,NaN
1,173433,318,21000716,indication,Gout,C0018099,Gout,T047,90560007,benzbromarone,D001553,D006073,NaN,NaN


In [108]:
# Do some rennaming of the columns before saving
rels = rels.rename(columns={'c_name': 'compound_name',
                            'concept_name': 'disease_name',
                            'compound_new_id': 'compound_semmed_id',
                            'disease_new_id': 'disease_semmed_id'})

In [109]:
# Only want indications for the gold standard
# Keep Duplicates in RELs just in case they're insightful, but indicaitons should have no dups.
inds = rels.query('relationship_name == "indication"').drop_duplicates(subset=['compound_semmed_id', 'disease_semmed_id'])

rels.to_csv('../data/gold_standard_relationships_nodemerge.csv', index=False)
inds.to_csv('../data/indications_nodemerge.csv', index=False)